<br>

<div align=center><font color=maroon size=6><b>Distributed training with Keras</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 official tutorials: <a href="https://www.tensorflow.org/tutorials" style="text-decoration:none;">TensorFlow Tutorials</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Tutorials > <a href="https://www.tensorflow.org/tutorials/distribute/keras" style="text-decoration:none;">Distributed training with Keras</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/keras.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

## Overview

The `tf.distribute.Strategy` API provides an abstraction for distributing your training across multiple processing units. It allows you to carry out distributed training using existing models and training code with minimal changes.

<font color=maroon size=3>This tutorial demonstrates how to use the `tf.distribute.MirroredStrategy` to perform in-graph replication with **_synchronous training on many GPUs on one machine_**. The strategy essentially copies all of the model's variables to each processor. Then, it uses [all-reduce](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) to combine the gradients from all processors, and applies the combined value to all copies of the model.</font>

You will use the `tf.keras` APIs to build the model and `Model.fit` for training it. <font color=maroon size=3>(To learn about distributed training with a custom training loop and the `MirroredStrategy`, check out [this tutorial](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/custom_training.ipynb).)</font>

<font color=maroon size=3>`MirroredStrategy` trains your model on multiple GPUs on a single machine. For _synchronous training on many GPUs on multiple workers_, use the `tf.distribute.MultiWorkerMirroredStrategy` with the [Keras Model.fit](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb) or [a custom training loop](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_ctl.ipynb). For other options, refer to the [Distributed training guide](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/distributed_training.ipynb).</font>

To learn about various other strategies, there is the [Distributed training with TensorFlow](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/distributed_training.ipynb) guide.

<br>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import os

D:\Programs\Anaconda3\envs\tfg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tf.__version__

'2.8.0'

<br>

In [3]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

<br>
<br>
<br>

## Download the dataset

Load the MNIST dataset from [TensorFlow Datasets](https://www.tensorflow.org/datasets). This returns a dataset in the `tf.data` format.

Setting the `with_info` argument to `True` includes the metadata for the entire dataset, which is being saved here to `info`. Among other things, this metadata object includes the number of train and test examples.

In [4]:
datasets, info = tfds.load(name='mnist',
                           with_info=True, 
                           as_supervised=True,
                           data_dir="D:/KeepStudy/0_Coding/0_dataset/")

<br>

In [5]:
mnist_train, mnist_test = datasets['train'], datasets['test']

<br>

In [6]:
datasets

{'train': <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [7]:
type(datasets)

dict

In [8]:
mnist_train

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [9]:
type(mnist_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

<br>
<br>
<br>

## Define the distribution strategy

Create a `MirroredStrategy` object. This will handle distribution and provide a context manager (`MirroredStrategy.scope`) to build your model inside.

In [10]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [11]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


<br>
<br>
<br>

## Set up the input pipeline

<font size=3>When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. <font color=maroon>In general, use the largest batch size that fits the GPU memory and tune the learning rate accordingly.</font></font>

In [12]:
info.splits

{'train': <SplitInfo num_examples=60000, num_shards=1>,
 'test': <SplitInfo num_examples=10000, num_shards=1>}

In [13]:
info.splits.total_num_examples

70000

In [14]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

<br>

Define a function that normalizes the image pixel values from the `[0, 255]` range to the  `[0, 1]` range ([feature scaling](https://en.wikipedia.org/wiki/Feature_scaling)):

In [15]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

<br>

Apply this `scale` function to the training and test data, and then use the `tf.data.Dataset` APIs to shuffle the training data (`Dataset.shuffle`), and batch it (`Dataset.batch`). Notice that you are also keeping an in-memory cache of the training data to improve performance (`Dataset.cache`).

In [16]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [17]:
# help(mnist_train.shuffle)

<br>
<br>
<br>

## Create the model

Within the context of `Strategy.scope`, create and compile the model using the Keras API:

In [18]:
with strategy.scope():
    model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, 3, 
                                                        activation='relu', 
                                                        input_shape=(28, 28, 1)),
                                 tf.keras.layers.MaxPooling2D(),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(64, activation='relu'),
                                 tf.keras.layers.Dense(10)
                                ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


<br>
<br>
<br>

## Define the callbacks

Define the following [Keras Callbacks](https://www.tensorflow.org/guide/keras/train_and_evaluate):

- `tf.keras.callbacks.TensorBoard`: writes a log for TensorBoard, which allows you to visualize the graphs.
- `tf.keras.callbacks.ModelCheckpoint`: saves the model at a certain frequency, such as after every epoch.
- `tf.keras.callbacks.BackupAndRestore`: provides the fault tolerance functionality by backing up the model and current epoch number. Learn more in the _Fault tolerance_ section of the [Multi-worker training with Keras](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb) tutorial.
- `tf.keras.callbacks.LearningRateScheduler`: schedules the learning rate to change after, for example, every epoch/batch.

For illustrative purposes, add a [custom callback](https://www.tensorflow.org/guide/keras/custom_callback) called `PrintLR` to display the *learning rate* in the notebook.
<br>
<br>

<font size=3 color=maroon>**Note:** 

Use the `BackupAndRestore` callback instead of `ModelCheckpoint` as the main mechanism to restore the training state upon a restart from a job failure. Since `BackupAndRestore` only supports ***eager mode***, in ***graph mode*** consider using `ModelCheckpoint`.</font>

<br>

In [19]:
# Define the checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'

# Define the name of the checkpoint files.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

<br>

In [20]:
# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

<br>

In [21]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1, 
                                                          model.optimizer.lr.numpy())
             )

<br>

In [22]:
# Put all the callbacks together.
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs'),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                save_weights_only=True),
             tf.keras.callbacks.LearningRateScheduler(decay),
             PrintLR()
            ]

<br>
<br>
<br>

## Train and evaluate

Now, train the model in the usual way by calling Keras `Model.fit` on the model and passing in the dataset created at the beginning of the tutorial. This step is the same whether you are distributing the training or not.

In [23]:
EPOCHS = 12

model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/12
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


  1/938 [..............................] - ETA: 1:16:53 - loss: 2.3363 - accuracy: 0.1250WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0038s vs `on_train_batch_end` time: 0.0117s). Check your callbacks.


938/938 [==============================] - ETA: 0s - loss: 0.2036 - accuracy: 0.9399
Learning rate for epoch 1 is 0.0010000000474974513
938/938 [==============================] - 9s 4ms/step - loss: 0.2036 - accuracy: 0.9399 - lr: 0.0010
Epoch 2/12
925/938 [============================>.] - ETA: 0s - loss: 0.0665 - accuracy: 0.9807
Learning rate for epoch 2 is 0.0010000000474974513
938/938 [==============================] - 4s 4ms/step - loss: 0.0666 - accuracy: 0.9806 - lr: 0.0010
Epoch 3/12
926/938 [============================>.] - ETA: 0s - loss: 0.0475 - accuracy: 0.9856
Learning rate for epoch 3 is 0.0010000000474974513
938/938 [==============================] - 3s 4ms/step - loss: 0.0475 - accuracy: 0.9855 - lr: 0.0010
Epoch 4/12
929/938 [============================>.] - ETA: 0s - loss: 0.0265 - accuracy: 0.9925
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 [==============================] - 3s 4ms/step - loss: 0.0265 - accuracy: 0.9926 - lr: 1.0000e-04
Epoch 5/12


<br>

Check for saved checkpoints:

In [24]:
# Check the checkpoint directory.
!ls {checkpoint_dir}

checkpoint
ckpt_1.data-00000-of-00001
ckpt_1.index
ckpt_10.data-00000-of-00001
ckpt_10.index
ckpt_11.data-00000-of-00001
ckpt_11.index
ckpt_12.data-00000-of-00001
ckpt_12.index
ckpt_2.data-00000-of-00001
ckpt_2.index
ckpt_3.data-00000-of-00001
ckpt_3.index
ckpt_4.data-00000-of-00001
ckpt_4.index
ckpt_5.data-00000-of-00001
ckpt_5.index
ckpt_6.data-00000-of-00001
ckpt_6.index
ckpt_7.data-00000-of-00001
ckpt_7.index
ckpt_8.data-00000-of-00001
ckpt_8.index
ckpt_9.data-00000-of-00001
ckpt_9.index


<br>

To check how well the model performs, load the latest checkpoint and call `Model.evaluate` on the test data:

In [25]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 2s 5ms/step - loss: 0.0393 - accuracy: 0.9871
Eval loss: 0.039326261729002, Eval accuracy: 0.9871000051498413


<br>

To visualize the output, launch TensorBoard and view the logs:

In [26]:
%tensorboard --logdir=logs

Reusing TensorBoard on port 6006 (pid 12016), started 0:14:18 ago. (Use '!kill 12016' to kill it.)

<br>

In [27]:
!ls -sh ./logs

total 0
0 train


<br>
<br>
<br>

## Export to SavedModel

<font size=3 color=maroon>Export the graph and the variables to the platform-agnostic SavedModel format using Keras `Model.save`. After your model is saved, you can load it with or without the `Strategy.scope`.</font>

In [28]:
path = 'saved_model/'

In [29]:
model.save(path, save_format='tf')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


<br>

Now, load the model without `Strategy.scope`:

In [30]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy']
                          )

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 0s 2ms/step - loss: 0.0393 - accuracy: 0.9871
Eval loss: 0.039326261729002, Eval Accuracy: 0.9871000051498413


<br>

Load the model with `Strategy.scope`:

In [31]:
with strategy.scope():
    replicated_model = tf.keras.models.load_model(path)
    replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                             optimizer=tf.keras.optimizers.Adam(),
                             metrics=['accuracy'])

    eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
    print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 2s 3ms/step - loss: 0.0393 - accuracy: 0.9871
Eval loss: 0.039326261729002, Eval Accuracy: 0.9871000051498413


<br>
<br>
<br>

### Additional resources

More examples that use different distribution strategies with the Keras `Model.fit` API:

1. The [Solve GLUE tasks using BERT on TPU](https://www.tensorflow.org/text/tutorials/bert_glue) tutorial uses `tf.distribute.MirroredStrategy` for training on GPUs and `tf.distribute.TPUStrategy` on TPUs.
1. The [Save and load a model using a distribution strategy](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/save_and_load.ipynb) tutorial demonstates how to use the SavedModel APIs with `tf.distribute.Strategy`.
1. The [official TensorFlow models](https://github.com/tensorflow/models/tree/master/official) can be configured to run multiple distribution strategies.

To learn more about TensorFlow distribution strategies:

1. The [Custom training with tf.distribute.Strategy](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/custom_training.ipynb) tutorial shows how to use the `tf.distribute.MirroredStrategy` for single-worker training with a custom training loop.
1. The [Multi-worker training with Keras](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb) tutorial shows how to use the `MultiWorkerMirroredStrategy` with `Model.fit`.
1. The [Custom training loop with Keras and MultiWorkerMirroredStrategy](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_ctl.ipynb) tutorial shows how to use the `MultiWorkerMirroredStrategy` with Keras and a custom training loop.
1. The [Distributed training in TensorFlow](https://www.tensorflow.org/guide/distributed_training) guide provides an overview of the available distribution strategies.
1. The [Better performance with tf.function](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/function.ipynb) guide provides information about other strategies and tools, such as the [TensorFlow Profiler](https://github.com/tensorflow/docs/blob/master/site/en/guide/profiler.md) you can use to optimize the performance of your TensorFlow models.

Note: `tf.distribute.Strategy` is actively under development and TensorFlow will be adding more examples and tutorials in the near future. Please give it a try. Your feedback is welcome—feel free to submit it via [issues on GitHub](https://github.com/tensorflow/tensorflow/issues/new).

<br>

```python
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.
```

<br>
<br>
<br>